In [ ]:
pip install flask
pip install -q -U google-genai
pip install pandas
pip install joblib
pip install uvicorn

In [1]:
print("\n --- Gemini AI for full credit confidence score prompt")
system_prompt = (
    "You are an expert AI credit scoring engine that performs comprehensive creditworthiness assessments for specific loan applications. You analyze demographic, financial, and banking features alongside ML-generated credit scores and loan-specific parameters to produce professional credit evaluations.\n"
    
    "CRITICAL: Your response must ONLY contain the structured output format below. Do not include explanations, reasoning, disclaimers, or additional text.\n\n"

    "## Required Output Format (EXACT):\n\n"

    "**🎯 Credit Confidence Score**\n"
    "**[SCORE]/100**\n"
    "**[RISK_LEVEL]**\n\n"

    "**Payment History (30%)**\n"
    "**[SUBSCORE]/30** - [Brief status]\n\n"

    "**Debt Management (25%)**\n"
    "**[SUBSCORE]/25** - [Brief status]\n\n"

    "**Savings Behavior (20%)**\n"
    "**[SUBSCORE]/20** - [Brief status]\n\n"

    "**Income Stability (15%)**\n"
    "**[SUBSCORE]/15** - [Brief status]\n\n"

    "**Account Management (10%)**\n"
    "**[SUBSCORE]/10** - [Brief status]\n\n"

    "**Risk Assessment:** [Single concise sentence about overall financial discipline and creditworthiness]\n\n"

    "## Scoring Guidelines:\n\n"

    "**Credit Confidence Score (0-100):**\n"
    "- 85-100: Excellent creditworthiness\n"
    "- 70-84: Good creditworthiness\n"
    "- 55-69: Fair creditworthiness\n"
    "- 40-54: Poor creditworthiness\n"
    "- 0-39: Very poor creditworthiness\n\n"

    "**Risk Labels:**\n"
    "- LOW RISK: 70-100\n"
    "- MEDIUM RISK: 40-69\n"
    "- HIGH RISK: 0-39\n\n"

    "**Category Assessment Criteria:**\n\n"

    "**Payment History (30% weight):**\n"
    "- Analyze payment patterns, defaults, late payments\n"
    "- Consider ML credit score impact on payment reliability\n"
    "- Score ranges: 24-30 (Excellent), 18-23 (Good), 12-17 (Fair), 6-11 (Poor), 0-5 (Very Poor)\n\n"

    "**Debt Management (25% weight):**\n"
    "- Evaluate debt-to-income ratio, credit utilization, overdrafts\n"
    "- Assess existing loan obligations and management\n"
    "- **CRITICAL**: Factor in requested loan amount vs income (debt-to-income impact)\n"
    "- **CRITICAL**: Consider monthly payment affordability vs current income\n"
    "- **CRITICAL**: Evaluate total repayment burden vs financial capacity\n"
    "- Score ranges: 20-25 (Excellent), 15-19 (Good), 10-14 (Fair), 5-9 (Poor), 0-4 (Very Poor)\n\n"

    "**Savings Behavior (20% weight):**\n"
    "- Review savings rate, emergency fund, financial reserves\n"
    "- Consider savings consistency and growth patterns\n"
    "- Score ranges: 16-20 (Excellent), 12-15 (Good), 8-11 (Fair), 4-7 (Poor), 0-3 (Very Poor)\n\n"

    "**Income Stability (15% weight):**\n"
    "- Assess employment history, income consistency, job security\n"
    "- Factor in income sources and reliability\n"
    "- **CRITICAL**: Evaluate income adequacy for requested loan payments\n"
    "- **CRITICAL**: Consider payment-to-income ratio sustainability\n"
    "- Score ranges: 12-15 (Excellent), 9-11 (Good), 6-8 (Fair), 3-5 (Poor), 0-2 (Very Poor)\n\n"

    "**Account Management (10% weight):**\n"
    "- Review banking relationship, account maintenance, financial behavior\n"
    "- Consider account age, transaction patterns, service issues\n"
    "- Score ranges: 8-10 (Excellent), 6-7 (Good), 4-5 (Fair), 2-3 (Poor), 0-1 (Very Poor)\n\n"

    "## Input Data Expected:\n"
    "- **Applicant Features**: Demographics, financial history, banking data\n"
    "- **ML Prediction Score**: Numerical score (e.g., ML model credit score = 605.08)\n"
    "- **Loan Details**:\n"
    "  - Loan amount requested\n"
    "  - Interest rate\n"
    "  - Payback period (months)\n"
    "  - Total repayment amount\n"
    "  - Monthly payment amount (if provided)\n\n"

    "## ML Model Score Integration:\n\n"

    "**Input Format:** You will receive an ML prediction score (e.g., ML model credit score = 605.08)\n\n"

    "**ML Score Interpretation:**\n"
    "- 750+: Excellent creditworthiness (strong positive indicator)\n"
    "- 650-749: Good creditworthiness (positive indicator)\n"
    "- 550-649: Fair creditworthiness (neutral to slight positive)\n"
    "- 450-549: Poor creditworthiness (negative indicator)\n"
    "- Below 450: Very poor creditworthiness (strong negative indicator)\n\n"

    "**Integration Method:**\n"
    "- Use ML score as a baseline indicator (40% weight)\n"
    "- Combine with feature analysis (60% weight) to generate final confidence score\n"
    "- ML score should influence both overall confidence and individual category assessments\n"
    "- Higher ML scores should positively impact Payment History and Debt Management categories\n"
    "- Lower ML scores should trigger more conservative risk assessments\n\n"

    "## Key Loan Assessment Factors:\n\n"

    "**Affordability Analysis:**\n"
    "- Monthly payment should not exceed 25-30% of monthly income\n"
    "- Total debt service ratio including new loan should stay under 40%\n"
    "- Loan term should be reasonable for loan purpose and amount\n\n"

    "**Risk Indicators:**\n"
    "- HIGH RISK: Monthly payment >40% of income, loan amount >5x monthly income\n"
    "- MEDIUM RISK: Monthly payment 30-40% of income, moderate debt load\n"
    "- LOW RISK: Monthly payment <25% of income, strong debt capacity\n\n"

    "**Interest Rate Consideration:**\n"
    "- Higher rates may indicate higher risk assessment is needed\n"
    "- Evaluate if applicant can handle rate increases (if variable)\n\n"

    "## Processing Instructions:\n"
    "1. Convert ML prediction score to creditworthiness indicator\n"
    "2. Analyze provided demographic/financial features\n"
    "3. **CRITICAL**: Calculate loan affordability metrics (payment-to-income, debt-to-income)\n"
    "4. **CRITICAL**: Assess loan amount reasonableness vs applicant profile\n"
    "5. Weight ML score (40%) with feature analysis (60%)\n"
    "6. Calculate category scores considering both ML score and loan-specific risks\n"
    "7. Generate final confidence score (0-100 scale) factoring loan suitability\n"
    "8. Assign risk label based on final confidence score and loan parameters\n"
    "9. Format output exactly as specified above\n\n"

    "REMINDER: Return ONLY the structured assessment format. No additional text, explanations, or commentary."
)


 --- Gemini AI for full credit confidence score prompt


In [2]:
import json
from typing import Any, Dict

def make_role_prompt(prediction_score: float, features: Dict[str, Any]) -> str:
    """
    Constructs a role prompt for Gemini credit confidence assessment.

    Args:
        prediction_score (float): The ML model credit score for the applicant.
        features (Dict[str, Any]): The applicant's feature set.

    Returns:
        str: Formatted prompt string for Gemini.
    """
    features_str = json.dumps(features, indent=2)
    prompt = (
        "Applicant Data for Credit Confidence Assessment:\n\n"
        f"ML model credit score: {prediction_score}\n\n"
        "Features:\n"
        f"{features_str}\n"
    )
    return prompt

In [ ]:
"""Gemini API request initialising"""
from google import genai
from google.genai import types

GEMINI_API_KEY = "your_gemini_api_key"

client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = "gemini-2.5-pro"
chat_config = types.GenerateContentConfig(
    system_instruction=system_prompt,
)

chat = client.chats.create(
    model=MODEL_ID,
    config=chat_config,
)
def ai_credit_confidence_score(role_prompt: str) -> str:
    """
    Sends the role prompt to Gemini and returns the model's response.
    """
    try:
        response = chat.send_message(role_prompt)
        return response.text if hasattr(response, "text") else str(response)
    except Exception as e:
        return f"Error: {str(e)}"
    

In [ ]:
"""API endpoint for credit confidence Score engine"""
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional, List
import joblib
import pandas as pd

# Load the trained pipeline
model = joblib.load("recommender_model.joblib")

app = FastAPI()

# Define the input data schema
class ApplicantData(BaseModel):
    # personal information
    Age: int
    Income: float
    MonthsEmployed: int
    DTIRatio: float
    Education: str
    EmploymentType: str
    MaritalStatus: str
    HasMortgage: int
    HasDependents: int
    LoanPurpose: str
    HasCoSigner: int
     # Banking and Financial Behavior
    AvgMonthlyBalance: float
    AvgMonthlySavings: float
    NumOverdraftsLast12Months: int
    SavingsRate: float
    DepositFrequency: int
    LastMonthSpending: float
    MinBalanceLast6Months: float
    MaxBalanceLast6Months: float
    AccountFlags: List[str]
     # Loan Details
    LoanPurpose: str
    LoanAmount: float
    LoanTermMonths : int
    InterestRate: float
    MonthlyPayment: float
    TotalRepaymentAmount: Optional[float] = None

@app.post("/predict")
def predict(data: ApplicantData):
    # Convert input into a DataFrame with expected column names
    features = pd.DataFrame([data.model_dump()])

    # Make prediction
    prediction_score = float(model.predict(features)[0])
    features_dict = data.model_dump()
    role_prompt = make_role_prompt(prediction_score, features_dict)
    credit_confidence_score = ai_credit_confidence_score(role_prompt)

    return {
        "credit_score": credit_confidence_score
        }


In [ ]:
import uvicorn
import threading
import time

def run_server():
    uvicorn.run("__main__:app", host="127.0.0.1", port=5000, reload=False)

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(3)
print("FastAPI server started on http://127.0.0.1:5000")

INFO:     Started server process [12016]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


FastAPI server started on http://127.0.0.1:5000


INFO:     127.0.0.1:59366 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59397 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59440 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59905 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59927 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60097 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60197 - "POST /predict HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:60254 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60314 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60331 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60348 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60358 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60360 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60361 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60368 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60374 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60

In [8]:
"""Test data"""
import requests
import json
from IPython.display import display, Markdown

# Suppose credit_confidence_response is your Gemini markdown string

# Test data
test_data = {
    # Personal Information
    "Age": 30,
    "Income": 50000.0,
    "MonthsEmployed": 24,
    "DTIRatio": 0.3,
    "Education": "Undergraduate",
    "EmploymentType": "Salaried",
    "MaritalStatus": "Single",
    "HasMortgage": 0,
    "HasDependents": 0,
    "HasCoSigner": 0,
    
    # Banking and Financial Behavior
    "AvgMonthlyBalance": 3500.0,
    "AvgMonthlySavings": 500.0,
    "NumOverdraftsLast12Months": 1,
    "SavingsRate": 0.12,
    "DepositFrequency": 2,
    "LastMonthSpending": 2700.0,
    "MinBalanceLast6Months": 1800.0,
    "MaxBalanceLast6Months": 4200.0,
    "AccountFlags": ["No recent overdraft", "Active savings"],
    
    # Loan Details
    "LoanPurpose": "Debt Consolidation",
    "LoanAmount": 15000.0,
    "LoanTermMonths": 36,  # 3 years
    "InterestRate": 12.5,  # 12.5% annual interest rate
    "MonthlyPayment": 502.84,  # Calculated monthly payment
    "TotalRepaymentAmount": 18102.24  # Total amount to be repaid
}

# Make prediction request
response = requests.post("http://127.0.0.1:5000/predict", json=test_data)

if response.status_code == 200:
    result = response.json()
    print("Credit Confidence score:")
    display(Markdown(result['credit_score']))

else:
    print(f"Error: {response.status_code} - {response.text}")

Credit Confidence score:


**🎯 Credit Confidence Score**
**67/100**
**MEDIUM RISK**

**Payment History (30%)**
**16/30** - Fair reliability indicated by credit score.

**Debt Management (25%)**
**11/25** - High overall debt burden despite an affordable new monthly payment.

**Savings Behavior (20%)**
**18/20** - Excellent savings rate and consistent financial reserves.

**Income Stability (15%)**
**13/15** - Stable salaried employment with sufficient income for loan payments.

**Account Management (10%)**
**9/10** - Positive account history with healthy balances.

**Risk Assessment:** The applicant shows strong savings and income stability, but the high total debt burden after the loan presents a moderate credit risk.

In [ ]:
"""Test data"""
import requests
import json
from IPython.display import display, Markdown


# Test data
test_data_1 = {
    "Age": 22,
    "Income": 18000,
    "MonthsEmployed": 3,
    "DTIRatio": 0.85,
    "Education": "High School",
    "EmploymentType": "Part-time",
    "MaritalStatus": "Single",
    "HasMortgage": 0,
    "HasDependents": 0,
    "HasCoSigner": 0,
    "AvgMonthlyBalance": 250,
    "AvgMonthlySavings": 20,
    "NumOverdraftsLast12Months": 8,
    "SavingsRate": 0.013,
    "DepositFrequency": 2,
    "LastMonthSpending": 1400,
    "MinBalanceLast6Months": 50,
    "MaxBalanceLast6Months": 350,
    "AccountFlags": ["Frequent overdrafts", "Low balance alerts"],
    "LoanPurpose": "Consumer goods",
    "LoanAmount": 1500,
    "LoanTermMonths": 12,
    "InterestRate": 0.24,
    "MonthlyPayment": 142.32,
    "TotalRepaymentAmount": 1707.84
}
# Make prediction request
response = requests.post("http://127.0.0.1:5000/predict", json=test_data_1)

if response.status_code == 200:
    result = response.json()
    print("Credit Confidence score:")
    display(Markdown(result['credit_score']))

else:
    print(f"Error: {response.status_code} - {response.text}")

Credit Confidence score:


**🎯 Credit Confidence Score**
**11/100**
**HIGH RISK**

**Payment History (30%)**
**5/30** - Very poor, evidenced by numerous overdrafts.

**Debt Management (25%)**
**0/25** - Unsustainable debt level makes new loan repayment highly improbable.

**Savings Behavior (20%)**
**2/20** - Negligible savings and no financial buffer for emergencies.

**Income Stability (15%)**
**3/15** - Very low and unstable income from recent part-time employment.

**Account Management (10%)**
**1/10** - Extremely poor financial discipline with frequent low balances.

**Risk Assessment:** The applicant's extremely high debt-to-income ratio, poor financial management, and unstable employment create an exceptionally high risk of default.

In [ ]:
"""Test data"""
import requests
import json
from IPython.display import display, Markdown


# Test data
test_data_2 = {
    "Age": 53,
    "Income": 22000,
    "MonthsEmployed": 10,
    "DTIRatio": 0.60,
    "Education": "High School",
    "EmploymentType": "Part-time",
    "MaritalStatus": "Divorced",
    "HasMortgage": 0,
    "HasDependents": 1,
    "HasCoSigner": 0,
    "AvgMonthlyBalance": 1200,
    "AvgMonthlySavings": 80,
    "NumOverdraftsLast12Months": 4,
    "SavingsRate": 0.0036,
    "DepositFrequency": 1,
    "LastMonthSpending": 1150,
    "MinBalanceLast6Months": 950,
    "MaxBalanceLast6Months": 1300,
    "AccountFlags": ["Basic account", "Recent overdraft"],
    "LoanPurpose": "Medical emergency",
    "LoanAmount": 2500,
    "LoanTermMonths": 18,
    "InterestRate": 0.19,
    "MonthlyPayment": 162.78,
    "TotalRepaymentAmount": 2930.04
}
# Make prediction request
response = requests.post("http://127.0.0.1:5000/predict", json=test_data_2)

if response.status_code == 200:
    result = response.json()
    print("Credit Confidence score:")
    display(Markdown(result['credit_score']))

else:
    print(f"Error: {response.status_code} - {response.text}")

Credit Confidence score:


**🎯 Credit Confidence Score**
**21/100**
**HIGH RISK**

**Payment History (30%)**
**8/30** - Poor reliability due to multiple recent overdrafts.

**Debt Management (25%)**
**2/25** - Extremely high existing debt makes the new loan unaffordable.

**Savings Behavior (20%)**
**5/20** - Poor savings rate provides virtually no buffer for emergencies.

**Income Stability (15%)**
**4/15** - Low and unstable income from short-term part-time employment.

**Account Management (10%)**
**2/10** - Poor financial discipline demonstrated by frequent overdrafts.

**Risk Assessment:** The applicant's extremely high pre-existing debt, combined with unstable income and poor account management, presents a very high risk of default.

In [19]:
"""Complete Credit Confidence Score Test Runner"""
import requests
import json
from IPython.display import display, Markdown

# All test data scenarios
test_scenarios = {
    "high_risk_young": {
        "Age": 22,
        "Income": 18000,
        "MonthsEmployed": 3,
        "DTIRatio": 0.85,
        "Education": "High School",
        "EmploymentType": "Part-time",
        "MaritalStatus": "Single",
        "HasMortgage": 0,
        "HasDependents": 0,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 250,
        "AvgMonthlySavings": 20,
        "NumOverdraftsLast12Months": 8,
        "SavingsRate": 0.013,
        "DepositFrequency": 2,
        "LastMonthSpending": 1400,
        "MinBalanceLast6Months": 50,
        "MaxBalanceLast6Months": 350,
        "AccountFlags": ["Frequent overdrafts", "Low balance alerts"],
        "LoanPurpose": "Consumer goods",
        "LoanAmount": 1500,
        "LoanTermMonths": 12,
        "InterestRate": 0.24,
        "MonthlyPayment": 142.32,
        "TotalRepaymentAmount": 1707.84
    },
    
    "low_risk_professional": {
        "Age": 35,
        "Income": 75000,
        "MonthsEmployed": 60,
        "DTIRatio": 0.25,
        "Education": "Bachelor's Degree",
        "EmploymentType": "Full-time",
        "MaritalStatus": "Married",
        "HasMortgage": 1,
        "HasDependents": 2,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 8500,
        "AvgMonthlySavings": 1200,
        "NumOverdraftsLast12Months": 0,
        "SavingsRate": 0.192,
        "DepositFrequency": 2,
        "LastMonthSpending": 4800,
        "MinBalanceLast6Months": 6200,
        "MaxBalanceLast6Months": 12000,
        "AccountFlags": ["Premium account", "High balance"],
        "LoanPurpose": "Home improvement",
        "LoanAmount": 15000,
        "LoanTermMonths": 36,
        "InterestRate": 0.08,
        "MonthlyPayment": 469.70,
        "TotalRepaymentAmount": 16909.20
    },
    
    "medium_risk_graduate": {
        "Age": 26,
        "Income": 42000,
        "MonthsEmployed": 8,
        "DTIRatio": 0.45,
        "Education": "Bachelor's Degree",
        "EmploymentType": "Full-time",
        "MaritalStatus": "Single",
        "HasMortgage": 0,
        "HasDependents": 0,
        "HasCoSigner": 1,
        "AvgMonthlyBalance": 2100,
        "AvgMonthlySavings": 300,
        "NumOverdraftsLast12Months": 2,
        "SavingsRate": 0.086,
        "DepositFrequency": 2,
        "LastMonthSpending": 2800,
        "MinBalanceLast6Months": 1200,
        "MaxBalanceLast6Months": 3200,
        "AccountFlags": ["New customer", "Student loan payments"],
        "LoanPurpose": "Debt consolidation",
        "LoanAmount": 8000,
        "LoanTermMonths": 24,
        "InterestRate": 0.14,
        "MonthlyPayment": 378.33,
        "TotalRepaymentAmount": 9079.92
    },
    
    "high_risk_senior": {
        "Age": 68,
        "Income": 28000,
        "MonthsEmployed": 0,
        "DTIRatio": 0.70,
        "Education": "High School",
        "EmploymentType": "Retired",
        "MaritalStatus": "Widowed",
        "HasMortgage": 1,
        "HasDependents": 0,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 3200,
        "AvgMonthlySavings": 150,
        "NumOverdraftsLast12Months": 1,
        "SavingsRate": 0.064,
        "DepositFrequency": 1,
        "LastMonthSpending": 2100,
        "MinBalanceLast6Months": 2800,
        "MaxBalanceLast6Months": 4500,
        "AccountFlags": ["Fixed income", "Senior account"],
        "LoanPurpose": "Medical emergency",
        "LoanAmount": 5000,
        "LoanTermMonths": 30,
        "InterestRate": 0.22,
        "MonthlyPayment": 203.85,
        "TotalRepaymentAmount": 6115.50
    },
    
    "ultra_low_risk": {
        "Age": 42,
        "Income": 120000,
        "MonthsEmployed": 120,
        "DTIRatio": 0.15,
        "Education": "Master's Degree",
        "EmploymentType": "Full-time",
        "MaritalStatus": "Married",
        "HasMortgage": 1,
        "HasDependents": 3,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 25000,
        "AvgMonthlySavings": 2500,
        "NumOverdraftsLast12Months": 0,
        "SavingsRate": 0.25,
        "DepositFrequency": 2,
        "LastMonthSpending": 7500,
        "MinBalanceLast6Months": 18000,
        "MaxBalanceLast6Months": 32000,
        "AccountFlags": ["Private banking", "Investment portfolio"],
        "LoanPurpose": "Investment property",
        "LoanAmount": 50000,
        "LoanTermMonths": 60,
        "InterestRate": 0.05,
        "MonthlyPayment": 943.56,
        "TotalRepaymentAmount": 56613.60
    },
    
    "gig_worker": {
        "Age": 29,
        "Income": 35000,
        "MonthsEmployed": 18,
        "DTIRatio": 0.55,
        "Education": "Some College",
        "EmploymentType": "Self-employed",
        "MaritalStatus": "Single",
        "HasMortgage": 0,
        "HasDependents": 1,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 1800,
        "AvgMonthlySavings": 200,
        "NumOverdraftsLast12Months": 3,
        "SavingsRate": 0.069,
        "DepositFrequency": 4,
        "LastMonthSpending": 2600,
        "MinBalanceLast6Months": 400,
        "MaxBalanceLast6Months": 4200,
        "AccountFlags": ["Irregular income", "Gig worker"],
        "LoanPurpose": "Vehicle repair",
        "LoanAmount": 3500,
        "LoanTermMonths": 24,
        "InterestRate": 0.18,
        "MonthlyPayment": 170.24,
        "TotalRepaymentAmount": 4085.76
    },
    
    "rebuilding_credit": {
        "Age": 45,
        "Income": 48000,
        "MonthsEmployed": 24,
        "DTIRatio": 0.40,
        "Education": "Bachelor's Degree",
        "EmploymentType": "Full-time",
        "MaritalStatus": "Divorced",
        "HasMortgage": 0,
        "HasDependents": 2,
        "HasCoSigner": 1,
        "AvgMonthlyBalance": 2800,
        "AvgMonthlySavings": 400,
        "NumOverdraftsLast12Months": 1,
        "SavingsRate": 0.10,
        "DepositFrequency": 2,
        "LastMonthSpending": 3200,
        "MinBalanceLast6Months": 1500,
        "MaxBalanceLast6Months": 4200,
        "AccountFlags": ["Credit rebuilding", "Previous bankruptcy"],
        "LoanPurpose": "Education/Training",
        "LoanAmount": 6000,
        "LoanTermMonths": 36,
        "InterestRate": 0.16,
        "MonthlyPayment": 206.32,
        "TotalRepaymentAmount": 7427.52
    },
    
    "first_time_borrower": {
        "Age": 24,
        "Income": 38000,
        "MonthsEmployed": 12,
        "DTIRatio": 0.30,
        "Education": "Bachelor's Degree",
        "EmploymentType": "Full-time",
        "MaritalStatus": "Single",
        "HasMortgage": 0,
        "HasDependents": 0,
        "HasCoSigner": 1,
        "AvgMonthlyBalance": 3500,
        "AvgMonthlySavings": 500,
        "NumOverdraftsLast12Months": 0,
        "SavingsRate": 0.158,
        "DepositFrequency": 2,
        "LastMonthSpending": 2200,
        "MinBalanceLast6Months": 2800,
        "MaxBalanceLast6Months": 4500,
        "AccountFlags": ["New customer", "No credit history"],
        "LoanPurpose": "First car",
        "LoanAmount": 12000,
        "LoanTermMonths": 48,
        "InterestRate": 0.12,
        "MonthlyPayment": 316.43,
        "TotalRepaymentAmount": 15188.64
    },
    
    "emergency_borrower": {
        "Age": 37,
        "Income": 32000,
        "MonthsEmployed": 36,
        "DTIRatio": 0.75,
        "Education": "High School",
        "EmploymentType": "Full-time",
        "MaritalStatus": "Married",
        "HasMortgage": 1,
        "HasDependents": 3,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 450,
        "AvgMonthlySavings": 50,
        "NumOverdraftsLast12Months": 6,
        "SavingsRate": 0.019,
        "DepositFrequency": 2,
        "LastMonthSpending": 2800,
        "MinBalanceLast6Months": 200,
        "MaxBalanceLast6Months": 800,
        "AccountFlags": ["Multiple overdrafts", "Emergency borrower"],
        "LoanPurpose": "Family emergency",
        "LoanAmount": 2000,
        "LoanTermMonths": 15,
        "InterestRate": 0.28,
        "MonthlyPayment": 161.33,
        "TotalRepaymentAmount": 2419.95
    },
    
    "business_owner": {
        "Age": 39,
        "Income": 65000,
        "MonthsEmployed": 84,
        "DTIRatio": 0.35,
        "Education": "Bachelor's Degree",
        "EmploymentType": "Self-employed",
        "MaritalStatus": "Married",
        "HasMortgage": 1,
        "HasDependents": 2,
        "HasCoSigner": 0,
        "AvgMonthlyBalance": 8200,
        "AvgMonthlySavings": 800,
        "NumOverdraftsLast12Months": 0,
        "SavingsRate": 0.148,
        "DepositFrequency": 3,
        "LastMonthSpending": 4200,
        "MinBalanceLast6Months": 5500,
        "MaxBalanceLast6Months": 12000,
        "AccountFlags": ["Business account", "Variable income"],
        "LoanPurpose": "Business expansion",
        "LoanAmount": 25000,
        "LoanTermMonths": 42,
        "InterestRate": 0.11,
        "MonthlyPayment": 692.86,
        "TotalRepaymentAmount": 29100.12
    }
}

def test_scenario():
    """Test each scenario individually"""
    results = {}
    print("=== SCENARIO TESTING ===\n")
    
    for scenario_name, test_data in test_scenarios.items():
        try:
            print(f"Testing: {scenario_name.replace('_', ' ').title()}")
            print("-" * 50)
            
            response = requests.post("http://127.0.0.1:5000/predict", json=test_data)
            
            if response.status_code == 200:
                result = response.json()
                results[scenario_name] = result
                
                # Display results
                print(f"Credit Confidence Score: {result.get('credit_score', 'N/A')}")
                if 'explanation' in result:
                    print(f"Explanation: {result['explanation']}")
                if 'risk_level' in result:
                    print(f"Risk Level: {result['risk_level']}")
                    
            else:
                print(f"❌ Error: {response.status_code} - {response.text}")
                results[scenario_name] = {"error": f"{response.status_code}: {response.text}"}
                
        except Exception as e:
            print(f"❌ Exception: {e}")
            results[scenario_name] = {"error": str(e)}
            
        print("\n" + "="*60 + "\n")
    
    return results

results = test_scenario()


=== SCENARIO TESTING ===

Testing: High Risk Young
--------------------------------------------------
Credit Confidence Score: **🎯 Credit Confidence Score**
**11/100**
**HIGH RISK**

**Payment History (30%)**
**5/30** - Very poor, evidenced by numerous overdrafts.

**Debt Management (25%)**
**0/25** - Unsustainable debt level makes new loan repayment highly improbable.

**Savings Behavior (20%)**
**2/20** - Negligible savings and no financial buffer for emergencies.

**Income Stability (15%)**
**3/15** - Very low and unstable income from recent part-time employment.

**Account Management (10%)**
**1/10** - Extremely poor financial discipline with frequent low balances.

**Risk Assessment:** The applicant's extremely high debt-to-income ratio, poor financial management, and unstable employment create an exceptionally high risk of default.


Testing: Low Risk Professional
--------------------------------------------------
Credit Confidence Score: **🎯 Credit Confidence Score**
**84/100**
